<a href="https://colab.research.google.com/github/ferkrum/web-scraper-clima-atual-vc-clima-historico/blob/main/Web_Scraper_Clima_Atual_vs_Clima_Hist%C3%B3rico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Web Scraper Clima Atual vs Clima Histórico

Como saber se a temperatura atual está acima ou abaixo da média? E a umidade? 

Para responder essa pergunta usei duas fontes de dados para obter essas informações e gerar essas comparações. 

Fonte de dados: 

URL = 'https://www.climatempo.com.br/previsao-do-tempo/agora/cidade/366/santamaria-rs'

URL2 = 'https://pt.climate-data.org/america-do-sul/brasil/rio-grande-do-sul/santa-maria-4498/#climate-graph'


Linguagem: Python

_libs: requests; Beautifulsoup, datetime, pytz timezone_

.

.

_...Próximos features:_ 
* criar um bot no telegram para receber essa informação.


In [ ]:
import requests 
from bs4 import BeautifulSoup as bs

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from datetime import datetime
from pytz import timezone 
brasil = timezone('Brazil/East')
bs_time = datetime.now(brasil)
timestampBrasil = bs_time.strftime('%Y-%m-%d_%H-%M-%S') 
#print(bs_time)
mes = bs_time.strftime('%m')
mes = int(mes)

Obtendo informações:

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}

#1ª fonte de dados: dados atualizados (Clima Tempo)

---

In [ ]:
URL = 'https://www.climatempo.com.br/previsao-do-tempo/agora/cidade/366/santamaria-rs'

In [ ]:
response = requests.get(URL, headers = headers).content

In [ ]:
soup = bs(response, 'html.parser')

In [ ]:
#obtem informacoes
temperaturaAtual = soup.find("div", class_= "_flex _justify-center _align-center").get_text(' ', strip = True)
descricaoAtual = soup.find("span", class_= "col").text

In [ ]:
#para obter sensacao termica atual
descricaoAtual2 = soup.find("div", class_= "no-gutters -gray _flex _justify-center _margin-t-20 _padding-b-20 _border-b-light-1")

In [ ]:
sensacaoAtual = descricaoAtual2.find("span", class_ = "").text

In [ ]:
detalhesAtual = soup.find("ul", class_= "variables-list _border-b-light-1").get_text(' ', strip = True)

In [ ]:
detalhesAtual

'Vento E - 13km/h Umidade 88% Pressão 1004hPa'

In [ ]:
detalhesAtual = detalhesAtual.replace('Vento', '').replace('Umidade', ',').replace('Pressão', ',').strip().split(',')
ventoAtual = detalhesAtual[0].strip()
umidadeAtual = detalhesAtual[1].strip()
pressaoAtual = detalhesAtual[2].strip()

#Criando Pandas Dataframe:
(Com dados do primeiro soup.find)

1. criar um dicionário com Arrays;
2. transformar em pd.DataFrame(dicionario)


In [ ]:
print ('Santa Maria - \t', timestampBrasil)
print('Temperatura:\t', temperaturaAtual)
print('Sensação:\t', sensacaoAtual[-3:])
print('Descrição:\t', descricaoAtual)
print('Vento:\t\t',ventoAtual)
print('Umidade:\t',umidadeAtual)
print('Pressão:\t',pressaoAtual)

Santa Maria - 	 2022-07-15_16-22-51
Temperatura:	 19º
Sensação:	 19°
Descrição:	 Muitas nuvens
Vento:		 E - 13km/h
Umidade:	 88%
Pressão:	 1004hPa


In [ ]:
dictCidade = {'Vento' : [ventoAtual],
              'Umidade' : [umidadeAtual],
              'Pressão' : [pressaoAtual]}

In [ ]:
df_dictCidade = pd.DataFrame(dictCidade)
df_dictCidade

,Vento,Umidade,Pressão
0,E - 13km/h,88%,1004hPa


#2ª fonte de dados: dados históricos

---


Adicionar comparativo com histórico de temperatura obtido em: 

https://pt.climate-data.org/america-do-sul/brasil/rio-grande-do-sul/santa-maria-4498/#climate-graph

In [ ]:
URL2 = 'https://pt.climate-data.org/america-do-sul/brasil/rio-grande-do-sul/santa-maria-4498/#climate-graph'

In [ ]:
response2 = requests.get(URL2, headers = headers).content

In [ ]:
soup2 = bs(response2, 'html.parser')

In [ ]:
tabela = soup2.find("table", id="weather_table").find_all("tr")

modelo do dicionario para Pandas DataFrame: 

```
df_dictCidadeHistorico = { "tempMedia" : [24.4, 24,1],
"tempMinima" : [12, 14],
"nomes" : ['janeiro', 'fevereiro']
}
```



In [ ]:
tabela

[<tr>
 <th> </th>
 <th>Janeiro</th>
 <th>Fevereiro</th>
 <th>Março</th>
 <th>Abril</th>
 <th>Maio</th>
 <th>Junho</th>
 <th>Julho</th>
 <th>Agosto</th>
 <th>Setembro</th>
 <th>Outubro</th>
 <th>Novembro</th>
 <th>Dezembro</th>
 </tr>, <tr>
 <td>Temperatura média (°C)</td>
 <td style="background: #FF6347 ;">
 24.4
 </td>
 <td style="background: #FF6347 ;">
 24.1
 </td>
 <td style="background: #FF8C00 ;">
 22.6
 </td>
 <td style="background: #FF8C00 ;">
 19.5
 </td>
 <td style="background: #FFA500 ;">
 15.7
 </td>
 <td style="background: #FFCC66 ;">
 14
 </td>
 <td style="background: #FFCC66 ;">
 13.1
 </td>
 <td style="background: #FFCC66 ;">
 14.8
 </td>
 <td style="background: #FFA500 ;">
 16.4
 </td>
 <td style="background: #FFA500 ;">
 18.9
 </td>
 <td style="background: #FF8C00 ;">
 20.9
 </td>
 <td style="background: #FF8C00 ;">
 23.4
 </td>
 </tr>, <tr>
 <td>Temperatura mínima (°C)</td>
 <td style="background: #FF8C00 ;">
 19.9
 </td>
 <td style="background: #FF8C00 ;">
 19.8
 </

In [ ]:
len(tabela)

8

In [ ]:
mesesTitulo = []
for i in tabela[0].find_all("th"):
  mesesTitulo.append(i.get_text(' ', strip=True))
mesesTitulo.pop(0)
mesesTitulo

colunasTabela = len(tabela[0].find_all("th"))
colunasTabela


linhasTabela = len(tabela) 
dadosTitulo = []
dadosTabela = []

for i in range(1, linhasTabela):   #percorre as linhas da tabela
  linhaTabela = []
  #print("===")
  tituloLinha = tabela[i].find_all("td")[0].get_text(' ', strip=True)
  #print(tituloLinha)
  dadosTitulo.append(tituloLinha)
  linhaTabela.append(tituloLinha)

  for j in range(1, colunasTabela):
    
    
    conteudoCelula = tabela[i].find_all("td")[j].get_text(' ', strip=True).replace('&percnt','')
    #print(conteudoCelula, ' linha', i, 'coluna', j)
    try:
      linhaTabela.append(int(conteudoCelula)) #armazena os dados das celulas no array linhaTabela
    except:
      linhaTabela.append(float(conteudoCelula)) #armazena os dados das celulas no array linhaTabela

  dadosTabela.append(linhaTabela)     #armazena o array linhaTabela no array dadosTabela

mesesTitulo.insert(0,'Meses')
dadosTabela.append(mesesTitulo)


In [ ]:
dadosTabela

[['Temperatura média (°C)',
  24.4,
  24.1,
  22.6,
  19.5,
  15.7,
  14,
  13.1,
  14.8,
  16.4,
  18.9,
  20.9,
  23.4],
 ['Temperatura mínima (°C)',
  19.9,
  19.8,
  18.3,
  15.4,
  11.9,
  10.2,
  8.9,
  10.3,
  11.8,
  14.4,
  15.8,
  18.3],
 ['Temperatura máxima (°C)',
  29.4,
  28.9,
  27.5,
  24.4,
  20.3,
  18.7,
  18.2,
  20.4,
  21.6,
  24,
  26.3,
  28.7],
 ['Chuva (mm)', 164, 154, 134, 159, 126, 136, 149, 123, 164, 215, 150, 164],
 ['Umidade(%)', 73, 76, 76, 79, 81, 83, 82, 79, 78, 78, 73, 71],
 ['Dias chuvosos (d)', 10, 10, 9, 8, 7, 7, 7, 7, 8, 9, 8, 9],
 ['Horas de sol (h)',
  9.2,
  8.7,
  7.6,
  7.0,
  6.0,
  5.9,
  6.4,
  7.1,
  6.9,
  7.4,
  8.9,
  9.7],
 ['Meses',
  'Janeiro',
  'Fevereiro',
  'Março',
  'Abril',
  'Maio',
  'Junho',
  'Julho',
  'Agosto',
  'Setembro',
  'Outubro',
  'Novembro',
  'Dezembro']]

In [ ]:
dictTemperaturas = {}

In [ ]:
for i in dadosTabela:
  indice = i.pop(0)
  dictTemperaturas[indice] = i

In [ ]:
dictTemperaturas

{'Chuva (mm)': [164, 154, 134, 159, 126, 136, 149, 123, 164, 215, 150, 164],
 'Dias chuvosos (d)': [10, 10, 9, 8, 7, 7, 7, 7, 8, 9, 8, 9],
 'Horas de sol (h)': [9.2,
  8.7,
  7.6,
  7.0,
  6.0,
  5.9,
  6.4,
  7.1,
  6.9,
  7.4,
  8.9,
  9.7],
 'Meses': ['Janeiro',
  'Fevereiro',
  'Março',
  'Abril',
  'Maio',
  'Junho',
  'Julho',
  'Agosto',
  'Setembro',
  'Outubro',
  'Novembro',
  'Dezembro'],
 'Temperatura máxima (°C)': [29.4,
  28.9,
  27.5,
  24.4,
  20.3,
  18.7,
  18.2,
  20.4,
  21.6,
  24,
  26.3,
  28.7],
 'Temperatura média (°C)': [24.4,
  24.1,
  22.6,
  19.5,
  15.7,
  14,
  13.1,
  14.8,
  16.4,
  18.9,
  20.9,
  23.4],
 'Temperatura mínima (°C)': [19.9,
  19.8,
  18.3,
  15.4,
  11.9,
  10.2,
  8.9,
  10.3,
  11.8,
  14.4,
  15.8,
  18.3],
 'Umidade(%)': [73, 76, 76, 79, 81, 83, 82, 79, 78, 78, 73, 71]}

In [ ]:
df_dictTemperaturas = pd.DataFrame(dictTemperaturas)

In [ ]:
df_dictTemperaturas

,Temperatura média (°C),Temperatura mínima (°C),Temperatura máxima (°C),Chuva (mm),Umidade(%),Dias chuvosos (d),Horas de sol (h),Meses
0,24.4,19.9,29.4,164,73,10,9.2,Janeiro
1,24.1,19.8,28.9,154,76,10,8.7,Fevereiro
2,22.6,18.3,27.5,134,76,9,7.6,Março
3,19.5,15.4,24.4,159,79,8,7.0,Abril
4,15.7,11.9,20.3,126,81,7,6.0,Maio
5,14.0,10.2,18.7,136,83,7,5.9,Junho
6,13.1,8.9,18.2,149,82,7,6.4,Julho
7,14.8,10.3,20.4,123,79,7,7.1,Agosto
8,16.4,11.8,21.6,164,78,8,6.9,Setembro
9,18.9,14.4,24.0,215,78,9,7.4,Outubro


## Agora, finalmente, podemos comparar o valor atual com os valores históricos obtidos:
**bold text**

In [ ]:
print ('Santa Maria - \t', timestampBrasil)
print('Temperatura:\t', temperaturaAtual)
print('Sensação:\t', sensacaoAtual[-3:])
print('Descrição:\t', descricaoAtual)
print('Vento:\t\t',ventoAtual)
print('Umidade:\t',umidadeAtual)
print('Pressão:\t',pressaoAtual)


Santa Maria - 	 2022-07-15_16-22-51
Temperatura:	 19º
Sensação:	 19°
Descrição:	 Muitas nuvens
Vento:		 E - 13km/h
Umidade:	 88%
Pressão:	 1004hPa


In [ ]:
consultaHistorico = df_dictTemperaturas.iloc[mes - 1]

# Resultados

In [ ]:
consultaHistorico

Temperatura média (°C)      13.1
Temperatura mínima (°C)      8.9
Temperatura máxima (°C)     18.2
Chuva (mm)                   149
Umidade(%)                    82
Dias chuvosos (d)              7
Horas de sol (h)             6.4
Meses                      Julho
Name: 6, dtype: object

In [ ]:
consultaHistorico['Umidade(%)']

82

In [ ]:
if (int(temperaturaAtual[:-1]) > consultaHistorico['Temperatura máxima (°C)']):
  print('A temperatura atual (',temperaturaAtual[:-1],') é MAIOR do que a média da temperatura máxima (',consultaHistorico['Temperatura máxima (°C)'],').')
elif (int(temperaturaAtual[:-1]) < consultaHistorico['Temperatura mínima (°C)']):
  print('A temperatura atual (',temperaturaAtual[:-1],')é MENOR do que a média da temperatura mínima (',consultaHistorico['Temperatura mínima (°C)'],').')

if (int(umidadeAtual[:-1]) > consultaHistorico['Umidade(%)']):
  print('A umidade atual (', umidadeAtual[:-1],') é MAIOR do que a média da umidade (', consultaHistorico['Umidade(%)'], ').')
else:
  print('A umidade atual (', umidadeAtual[:-1],') é MENOR do que a média da umidade (', consultaHistorico['Umidade(%)'], ').')


A temperatura atual ( 19 ) é MAIOR do que a média da temperatura máxima ( 18.2 ).
A umidade atual ( 88 ) é MAIOR do que a média da umidade ( 82 ).
